### Compiler for Regular Expressions to Non-Deterministic Finite Automata (NFA)
Currently set to only work on regular expressions 
with the alphabet of ['a','z'].

This works as a Left to Right compiler, giving precedence to the left
characters over the right.<br>
Of course this is the weakest form of 
precedence in the compiler, after the operator precedence.
<br>
#### <center>Operator Syntax</center>

|Precedence|Operator|Meaning|
|:-:|:-:|:-:|
|Highest|\*|Kleene Star|
|Middle|. (implied)|Concatenation|
|Lowest|\|Union|
|Any|( )|Paranthesis|

We can use a Stack Data Structure To implement Thompson's Algorithm of Recursive Computation of NFA from RE.

In [1]:
class Stack:
    def __init__(self):
        self.items = []
    def empty(self):
        return self.items == []
    def push(self, item):
        self.items.append(item)
    def pop(self):
        return self.items.pop()
    def peek(self):
        return self.items[len(self.items)-1]
    def size(self):
        return len(self.items)

`namedtuple` collection class can be used as an alternative to structures in C/C++.
In this case, The tuple represents a transition with the members `from_state`,`to_state` and `symbol`.

In [2]:
from tabulate import tabulate
from collections import namedtuple 
Transition = namedtuple('Transition', ['from_state', 'to_state', 'symbol'])

The following class methods can be used for performing Repeated Checks on input.

In [3]:
class check(object):    
    @staticmethod
    def ifAlpha(x):
        return True if x in 'abcdefghijklmnopqrstuvwxyz' else False
    @staticmethod
    def ifSymbol(x):
        return True if x in 'εabcdefghijklmnopqrstuvwxyz' else False
    @staticmethod
    def ifEpsilon(x):
        return True if x is 'ε' else False
    @staticmethod
    def ifOperator(x):
        return True if x in '()*|' else False   
    @staticmethod
    def ifRegexChar(x):
        return True if check.ifAlpha(x) or check.ifOperator(x) else False
    @staticmethod
    def ifRegex(re):
        if not re:
            return False
        for c in re:
            if not check.ifRegexChar(c):
                return False
            return True


### Thompson's Algorithm Implementation

The Below Class NFA Implements the Thompson Algorithm's Implementation of Regular Expression Evaluation to build the Required NFA.

#### Kleene Star Closure `(*)` Algorithm
1. Increase state size by 2 new ε transition states.
2. Add ε transition for new initial state.
3. Copy existing transitions into new NFA.
4. Update state numbers by 1 (+initial state).
5. Add ε transition from old final state to new final state.
6. Loop back from new final state to old initial state.
7. Add ε transition from new initial state to new final state.
8. Return generated NFA.

#### Concatenation `(.)` Algorithm
1. Delete nfa_to_concat's initial ε transition state.
2. Copy nfa_to_concat to original_nfa.
3. State size reduced by 1 to neglect nfa1 final ε transition state.
3. Combine nfa_to_concat and original_nfa after removing nfa_to_concat's initial state.
4. State size reduced by 1 for removing initial epsilon of nfa_to_concat.
5. State size reduced by 1 for removing final epsilon of original_nfa.
6. Return generated NFA.

#### Union `(|)` Algorithm
1. Increase state size by 1 for initial state of first_nfa and nfa_to_union.
2. Increase state size by 1 for final state of first_nfa and nfa_to_union.        
3. Branch from q0 to start of first_nfa.
4. Copy existing transitions of first_nfa.
5. Update state numbers by 1 (+initial state).
6. Add Transition from last first_nfa to the final state.
7. Branch from q0 to start of nfa_to_union.
8. Copy existing transitions of nfa_to_union.
9. Update state numbers by 1 (+initial state).
10. Add Transition from last nfa_to_union to the final state.
11. Return generated NFA.

In [4]:
class NFA(object):
    def __init__(self, size=None, symbol=None):
        if size:
            self.states = []
            self.transitions = []
            self.final_state = 0
            self.setStateSize(size)
        elif symbol:
            self.states = []
            self.transitions = []
            self.setStateSize(2)
            self.final_state = 1
            self.transitions.append(Transition(0,1,symbol))
        else:
            self.states = []
            self.transitions = []
            self.final_state = 0
    def setStateSize(self, size):
        for i in range(size):
            self.states.append(i)
    def show(self):
        print("\nDelta (δ) Transitions")
        for t in self.transitions:
            print("δ(q{0},{1}) → q{2}".format(t.from_state,t.symbol,t.to_state))
    def getFinal(self):
        return max([t[1] for t in self.transitions])
    def indicator(self, x):
        if x is 0:
            return '→'
        elif x is self.getFinal():
            return '*'
        else:
            return ' '
    def transition_table(self):
        table = []
        row = []
        headers = []
        final_state = self.getFinal()
        symbols = sorted(set([t[2] for t in self.transitions]))
        states = ['{0} q{1}'.format(self.indicator(x),x) for x in range(0,final_state+1)]
        print("\nTransition Table")
        headers.append('δ')
        for i in range(len(symbols)):
            headers.append(symbols[i])
        for i in range(0,final_state+1):
            row.append(states[i])
            for j in symbols:
                to_states = ['q'+str(t.to_state) for t in self.transitions if t.symbol == j and t.from_state == i]
                if not to_states:
                    row.append('Ø')
                else:
                    row.append("{"+','.join(to_states)+"}")
            table.append(row)
            row = []
        print(tabulate(table,headers,"fancy_grid"))
    @staticmethod
    def kleene(nfa):
        result = NFA(size=len(nfa.states)+2)
        result.transitions.append(Transition(0,1,'ε'))
        for t in nfa.transitions:
            result.transitions.append(Transition(
                t.from_state+1, t.to_state+1, t.symbol
            ))
        result.transitions.append(Transition(
            len(nfa.states), len(nfa.states)+1,'ε'
        ))
        result.transitions.append(Transition(
            len(nfa.states), 1,'ε'
        ))
        result.transitions.append(Transition(
            0, len(nfa.states)+1,'ε'
        ))        
        result.final_state = len(nfa.states)+1
        return result
    @staticmethod
    def concat (nfa1, nfa2):
        nfa2.states.remove(0)
        for t in nfa2.transitions:
            nfa1.transitions.append(Transition(
                t.from_state+len(nfa1.states)-1,
                t.to_state+len(nfa1.states)-1,
                t.symbol
            ))
        for s in nfa2.states:
            nfa1.states.append(s+len(nfa1.states)+1)
        nfa1.final_state = len(nfa1.states)+len(nfa2.states)-2
        return nfa1
    @staticmethod
    def union (nfa1, nfa2):
        result = NFA(size=len(nfa1.states)+len(nfa2.states)+2)
        result.transitions.append(Transition(0,1,'ε'))
        for t in nfa1.transitions:            
            result.transitions.append(Transition(
                t.from_state+1, t.to_state+1, t.symbol
            ))
        result.transitions.append(Transition(
            len(nfa1.states),
            len(nfa1.states)+len(nfa2.states)+1,
            'ε'))
        result.transitions.append(
            Transition(0,len(nfa1.states)+1,'ε'))
        for t in nfa2.transitions:
            result.transitions.append(Transition(
                t.from_state+len(nfa1.states)+1, 
                t.to_state+len(nfa1.states)+1, 
                t.symbol
            ))
        result.transitions.append(Transition(
            len(nfa2.states)+len(nfa1.states),
            len(nfa1.states)+len(nfa2.states)+1,
            'ε'))
        return result
    #################### COMPILER ####################
    @staticmethod
    def compiles(regex):
        import sys
        if not check.ifRegex(regex):
            raise ValueError("Invalid Regular Expression Input.")
            return NFA()
        operators = Stack()
        operands = Stack()
        concat_stack = Stack()
        ccflag = False
        op = ""
        c = ""
        para_count = 0
        nfa1 = NFA()
        nfa2 = NFA()
        for i in range(len(regex)):
            c = regex[i]
            if check.ifSymbol(c):
                operands.push(NFA(symbol=c))
                if (ccflag):
                    operators.push('.')
                else:
                    ccflag = True
            else:
                if c == ')':
                    ccflag = True
                    if para_count == 0:
                        raise ValueError("More end paranthesis than beginning paranthesis.")
                        sys.exit(1)
                    else:
                        para_count-=1
                    while not operators.empty() and operators.peek() != '(':
                        op = operators.pop()
                        if op == '.':
                            nfa2 = operands.pop()
                            nfa1 = operands.pop()
                            operands.push(NFA.concat(nfa1, nfa2))
                        elif op == '|':
                            nfa2 = operands.pop()
                            if not operators.empty() and operators.peek() == '.':
                                concat_stack.push(operands.pop())
                                while not operators.empty() and operators.peek() == '.':
                                    concat_stack.push(operands.pop())
                                    operators.pop()
                                nfa1 = NFA.concat(concat_stack.pop(), concat_stack.pop())
                                while concat_stack.size() > 0:
                                    nfa1 =  NFA.concat(nfa1, concat_stack.pop())
                            else:
                                nfa1 = operands.pop()
                            operands.push(NFA.union(nfa1, nfa2))
                elif c == '*':
                    operands.push(NFA.kleene(operands.pop()))
                    ccflag = True
                elif c == '(':
                    operators.push(c)
                    para_count += 1
                elif c == '|':
                    operators.push(c)
                    ccflag = False
        while operators.size() > 0:
            if operands.empty():
                raise ValueError("Unbalanced Operators.")
                sys.exit(1)
            op = operators.pop()
            if op == '.':
                nfa2 = operands.pop()
                nfa1 = operands.pop()
                operands.push(NFA.concat(nfa1, nfa2))
            elif op == '|':
                nfa2 = operands.pop();
                if not operators.empty() and operators.peek() == '.':
                    concat_stack.push(operands.pop())
                    while not operators.empty() and operators.peek() == '.':
                        concat_stack.push(operands.pop())
                        operators.pop()
                    nfa1 = NFA.concat(concat_stack.pop(),concat_stack.pop())
                    while concat_stack.size() > 0:
                        nfa1 = NFA.concat(nfa1, concat_stack.pop())
                else:
                    nfa1 = operands.pop()
                operands.push(NFA.union(nfa1, nfa2))
        return operands.pop()
    @classmethod
    def main(cls, args):
        running = True
        while running:
            re_to_nfa = None
            re_to_nfa = NFA.compiles(input("Enter Regular Expression: "))
            re_to_nfa.transition_table()
            re_to_nfa.show()
            choice = input("continue → [y│n]: ")
            if choice is 'n':
                running = False
            else:
                continue
        print("\nProcess Terminated.")
        return

In [5]:
if __name__ == '__main__':
    import sys
    NFA.main(sys.argv)

Enter Regular Expression: a

Transition Table
╒══════╤══════╕
│ δ    │ a    │
╞══════╪══════╡
│ → q0 │ {q1} │
├──────┼──────┤
│ * q1 │ Ø    │
╘══════╧══════╛

Delta (δ) Transitions
δ(q0,a) → q1
continue → [y│n]: y
Enter Regular Expression: a*

Transition Table
╒══════╤══════╤═════════╕
│ δ    │ a    │ ε       │
╞══════╪══════╪═════════╡
│ → q0 │ Ø    │ {q1,q3} │
├──────┼──────┼─────────┤
│ q1   │ {q2} │ Ø       │
├──────┼──────┼─────────┤
│ q2   │ Ø    │ {q3,q1} │
├──────┼──────┼─────────┤
│ * q3 │ Ø    │ Ø       │
╘══════╧══════╧═════════╛

Delta (δ) Transitions
δ(q0,ε) → q1
δ(q1,a) → q2
δ(q2,ε) → q3
δ(q2,ε) → q1
δ(q0,ε) → q3
continue → [y│n]: y
Enter Regular Expression: a|b

Transition Table
╒══════╤══════╤══════╤═════════╕
│ δ    │ a    │ b    │ ε       │
╞══════╪══════╪══════╪═════════╡
│ → q0 │ Ø    │ Ø    │ {q1,q3} │
├──────┼──────┼──────┼─────────┤
│ q1   │ {q2} │ Ø    │ Ø       │
├──────┼──────┼──────┼─────────┤
│ q2   │ Ø    │ Ø    │ {q5}    │
├──────┼──────┼──────┼─────────┤


### Thank you!
**By**  : *Arvind Srinivasan*<br>
**Reg.**: *RA1611003010415*<br>
**Sem.**: *VI, B.Tech., C.S.E.*